In [1]:
import numpy as NP
from astroutils import nonmathops as NMO
from ClosureInvariants import graphUtils as GU
from ClosureInvariants import vectorInvariants as VI

## Set up antennas and array info

In [2]:
element_ids = ['O', 'A', 'B', 'C', 'D', 'E', 'F', 'G']
nelements_total = len(element_ids)
elements_subset = ['O', 'A', 'C', 'E', 'G']
elements_remove = ['B', 'D', 'F']
nelements_subset = len(elements_subset)
element_indices_subset = NMO.find_list_in_list(element_ids, elements_subset)
print(element_indices_subset)

[0 1 3 5 7]


## Determine antenna pairs

In [3]:
element_pairs = [(element_ids[i], element_ids[j]) for i in range(len(element_ids)) for j in range(i+1,len(element_ids))]
npairs = len(element_pairs)
element_pairs_subset = [(elements_subset[i], elements_subset[j]) for i in range(len(elements_subset)) for j in range(i+1,len(elements_subset))]
npairs_subset = len(element_pairs_subset)
element_pairs_indices = [(i,j) for i in range(len(element_ids)) for j in range(i+1,len(element_ids))]
element_pairs_subset_indices = [(element_indices_subset[i],element_indices_subset[j]) for i in range(len(elements_subset)) for j in range(i+1,len(elements_subset))]
indices_of_subset_pairs = NMO.find_list_in_list(list(map(str,element_pairs)), list(map(str,element_pairs_subset))) # Indices of subset element pairs in all element pairs
print(element_pairs)
print(element_pairs_indices)
print(element_pairs_subset)
print(element_pairs_subset_indices)
print(indices_of_subset_pairs)

[('O', 'A'), ('O', 'B'), ('O', 'C'), ('O', 'D'), ('O', 'E'), ('O', 'F'), ('O', 'G'), ('A', 'B'), ('A', 'C'), ('A', 'D'), ('A', 'E'), ('A', 'F'), ('A', 'G'), ('B', 'C'), ('B', 'D'), ('B', 'E'), ('B', 'F'), ('B', 'G'), ('C', 'D'), ('C', 'E'), ('C', 'F'), ('C', 'G'), ('D', 'E'), ('D', 'F'), ('D', 'G'), ('E', 'F'), ('E', 'G'), ('F', 'G')]
[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 4), (3, 5), (3, 6), (3, 7), (4, 5), (4, 6), (4, 7), (5, 6), (5, 7), (6, 7)]
[('O', 'A'), ('O', 'C'), ('O', 'E'), ('O', 'G'), ('A', 'C'), ('A', 'E'), ('A', 'G'), ('C', 'E'), ('C', 'G'), ('E', 'G')]
[(0, 1), (0, 3), (0, 5), (0, 7), (1, 3), (1, 5), (1, 7), (3, 5), (3, 7), (5, 7)]
[0 2 4 6 8 10 12 19 21 26]


## Determine the complete and independent set of triads

In [4]:
baseid = 'O'
base_ind = element_ids.index(baseid)
base_ind_subset = elements_subset.index(baseid)
triads_indep = GU.generate_triads(element_ids, baseid=baseid)
triads_subset_indep = GU.generate_triads(elements_subset, baseid=baseid)
indices_subset_triads = NMO.find_list_in_list(list(map(str,triads_indep)), list(map(str,triads_subset_indep))) # Indices of subset triads in all triads
print(triads_indep)
print(triads_subset_indep)
print(indices_subset_triads)

[('O', 'A', 'B'), ('O', 'A', 'C'), ('O', 'A', 'D'), ('O', 'A', 'E'), ('O', 'A', 'F'), ('O', 'A', 'G'), ('O', 'B', 'C'), ('O', 'B', 'D'), ('O', 'B', 'E'), ('O', 'B', 'F'), ('O', 'B', 'G'), ('O', 'C', 'D'), ('O', 'C', 'E'), ('O', 'C', 'F'), ('O', 'C', 'G'), ('O', 'D', 'E'), ('O', 'D', 'F'), ('O', 'D', 'G'), ('O', 'E', 'F'), ('O', 'E', 'G'), ('O', 'F', 'G')]
[('O', 'A', 'C'), ('O', 'A', 'E'), ('O', 'A', 'G'), ('O', 'C', 'E'), ('O', 'C', 'G'), ('O', 'E', 'G')]
[1 3 5 12 14 19]


## Generate mock cross-correlations

In [5]:
nruns_shape = (16,12,10) # Shape of number of runs (for example, number of random realisations, timestamps, channels)
pol_axes = (-2,-1) # Polarisation axes
npol = len(pol_axes)
bl_axis = -3 # Baseline axis
xc_real_std = 1.0
xc_imag_std = 1.0
randseed = None
rng = NP.random.default_rng(randseed)
xcorr = rng.normal(loc=0.0, scale=xc_real_std, size=nruns_shape+(npairs,npol,npol)) + 1j * rng.normal(loc=0.0, scale=xc_imag_std, size=nruns_shape+(npairs,npol,npol)) # The last two are polarisation axes
xcorr_subset = NP.take(xcorr, indices_of_subset_pairs, axis=bl_axis) # Take a subset of the baseline axes based on the elements subset
print(xcorr.shape)
print(xcorr_subset.shape)

(16, 12, 10, 28, 2, 2)
(16, 12, 10, 10, 2, 2)


## Set up triangular loop correlations

In [6]:
corrs_lol = VI.corrs_list_on_loops(xcorr, element_pairs, triads_indep, bl_axis=bl_axis, pol_axes=pol_axes)
corrs_lol_subset_method1 = VI.corrs_list_on_loops(xcorr, element_pairs, triads_subset_indep, bl_axis=bl_axis, pol_axes=pol_axes) # Method 1 from all correlations
corrs_lol_subset_method2 = VI.corrs_list_on_loops(xcorr_subset, element_pairs_subset, triads_subset_indep, bl_axis=bl_axis, pol_axes=pol_axes) # Method 2 from subset correlations
print(len(corrs_lol))
print(len(corrs_lol_subset_method1))
print(len(corrs_lol_subset_method2))
print(NP.max(NP.abs(NP.array(corrs_lol_subset_method1)-NP.array(corrs_lol_subset_method2)))) # Verify that both methods of setup produce identical results

21
6
6
0.0


## Compute advariants

In [7]:
advariants = VI.advariants_multiple_loops(corrs_lol, pol_axes=pol_axes)
advariants_subset = VI.advariants_multiple_loops(corrs_lol_subset_method1, pol_axes=pol_axes)
print(advariants.shape)
print(advariants_subset.shape)

(16, 12, 10, 21, 2, 2)
(16, 12, 10, 6, 2, 2)


## Compute 4-vectors from advariants

In [8]:
z4 = VI.vector_from_advariant(advariants)
z4_subset = VI.vector_from_advariant(advariants_subset)
print(z4.shape)
print(z4_subset.shape)

(16, 12, 10, 21, 4)
(16, 12, 10, 6, 4)


## Compute Minkowski dot products from the complex 4-vectors

Number of Minkowski dot products = $4(5)/2 + 4*(2N_\Delta - 4) = 4N^2 - 12N + 2$, where, $N_\Delta = \frac{(N-1)(N-2)}{2}$ and $N$ is the number of elements

In [9]:
mdp = VI.complete_minkowski_dots(z4)
mdp_subset = VI.complete_minkowski_dots(z4_subset)
print(mdp.shape)
print(mdp_subset.shape)

(16, 12, 10, 162)
(16, 12, 10, 42)


## Compute the invariants by removing the unknown common scaling factor

In [10]:
cloinv = VI.remove_scaling_factor_minkoski_dots(mdp, wts=None)
cloinv_subset = VI.remove_scaling_factor_minkoski_dots(mdp_subset, wts=None)
print(cloinv.shape)
print(cloinv_subset.shape)

(16, 12, 10, 162)
(16, 12, 10, 42)


## Test the invariance of the closure invariants obtained by corrupting the correlations using element-based gains

## Generate mock polarimetric gains

In [11]:
element_axis = -3 # Antenna axis
mean_gain_scale = 3.0 
element_gains = rng.normal(loc=1.0, scale=NP.sqrt(0.5)/mean_gain_scale, size=nruns_shape+(nelements_total,npol,npol)).astype(NP.float64) + 1j * rng.normal(loc=1.0, scale=NP.sqrt(0.5)/mean_gain_scale, size=nruns_shape+(nelements_total,npol,npol)).astype(NP.float64) # shape is (...,n_element,2,2)
element_gains_subset = NP.take(element_gains, element_indices_subset, axis=element_axis)
print(element_gains.shape)
print(element_gains_subset.shape)

(16, 12, 10, 8, 2, 2)
(16, 12, 10, 5, 2, 2)


## Corrupt the correlations

In [12]:
prefactor_gains = NP.take(element_gains, NP.array(element_pairs_indices)[:,0], axis=element_axis) # A collection of g_a
postfactor_gains = NP.take(element_gains, NP.array(element_pairs_indices)[:,1], axis=element_axis) # A collection of g_b
xcorr_mod = VI.corrupt_visibilities(xcorr, prefactor_gains, postfactor_gains, pol_axes=pol_axes)

prefactor_gains_subset = NP.take(element_gains, NP.array(element_pairs_subset_indices)[:,0], axis=element_axis) # A collection of g_a
postfactor_gains_subset = NP.take(element_gains, NP.array(element_pairs_subset_indices)[:,1], axis=element_axis) # A collection of g_b
xcorr_subset_mod = VI.corrupt_visibilities(xcorr_subset, prefactor_gains_subset, postfactor_gains_subset, pol_axes=pol_axes)

print(xcorr_mod.shape)
print(xcorr_subset_mod.shape)

(16, 12, 10, 28, 2, 2)
(16, 12, 10, 10, 2, 2)


## Compute the closure invariants through advariants, 4-vectors, Minkowski dot products, and scale factor elimination

In [13]:
corrs_mod_lol = VI.corrs_list_on_loops(xcorr_mod, element_pairs, triads_indep, bl_axis=bl_axis, pol_axes=pol_axes)
corrs_mod_lol_subset_method1 = VI.corrs_list_on_loops(xcorr_mod, element_pairs, triads_subset_indep, bl_axis=bl_axis, pol_axes=pol_axes) # Method 1 from all correlations
corrs_mod_lol_subset_method2 = VI.corrs_list_on_loops(xcorr_subset_mod, element_pairs_subset, triads_subset_indep, bl_axis=bl_axis, pol_axes=pol_axes) # Method 2 from subset correlations
print(len(corrs_mod_lol))
print(len(corrs_mod_lol_subset_method1))
print(len(corrs_mod_lol_subset_method2))
print(NP.max(NP.abs(NP.array(corrs_mod_lol_subset_method1)-NP.array(corrs_mod_lol_subset_method2)))) # Verify that both methods of setup produce identical results

21
6
6
0.0


In [14]:
advariants_mod = VI.advariants_multiple_loops(corrs_mod_lol, pol_axes=pol_axes)
advariants_mod_subset = VI.advariants_multiple_loops(corrs_mod_lol_subset_method1, pol_axes=pol_axes)
print(advariants_mod.shape)
print(advariants_mod_subset.shape)

(16, 12, 10, 21, 2, 2)
(16, 12, 10, 6, 2, 2)


In [15]:
z4_mod = VI.vector_from_advariant(advariants_mod)
z4_mod_subset = VI.vector_from_advariant(advariants_mod_subset)
print(z4_mod.shape)
print(z4_mod_subset.shape)

(16, 12, 10, 21, 4)
(16, 12, 10, 6, 4)


In [16]:
mdp_mod = VI.complete_minkowski_dots(z4_mod)
mdp_mod_subset = VI.complete_minkowski_dots(z4_mod_subset)
print(mdp_mod.shape)
print(mdp_mod_subset.shape)

(16, 12, 10, 162)
(16, 12, 10, 42)


In [17]:
cloinv_mod = VI.remove_scaling_factor_minkoski_dots(mdp_mod, wts=None)
cloinv_mod_subset = VI.remove_scaling_factor_minkoski_dots(mdp_mod_subset, wts=None)
print(cloinv_mod.shape)
print(cloinv_mod_subset.shape)

(16, 12, 10, 162)
(16, 12, 10, 42)


## Check for invariance between ideal and modified versions

In [18]:
print(NP.allclose(cloinv, cloinv_mod))
print(NP.allclose(cloinv_subset, cloinv_mod_subset))

True
True


## Verify scale factors are as expected, namely absolute squared value of the determinant of the polarimetric gain of the base vertex

In [19]:
scale_factor = NP.abs(NP.linalg.det(element_gains))**2
scale_factor_subset = NP.abs(NP.linalg.det(element_gains_subset))**2

In [20]:
print(NP.allclose(scale_factor[...,[base_ind]], mdp_mod / mdp))
print(NP.allclose(scale_factor_subset[...,[base_ind_subset]], mdp_mod_subset / mdp_subset))

True
True
